In [1]:
import cv2
import os
import re
from tqdm import tqdm

## Split videos into frames

In [2]:
def getFrame(sec, count, vidcap):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()
    if hasFrames:
        cv2.imwrite(os.path.join("videos", "frame_split_output", filename+"_%d.jpg" % count), image)     # save frame as JPG file
    return hasFrames

In [3]:
def split_into_frames(filename, fps):
    vidcap = cv2.VideoCapture(os.path.join("videos", "video_input", filename+'.mp4'))
    sec = 0
    count = 1
    success = getFrame(sec, count, vidcap)
    while success:
        count += 1
        sec = round(sec+(1/fps), 2)
        success = getFrame(sec, count, vidcap)
        if success == False:
            print("Done")

In [4]:
filename = "ocean"
fps = 25
split_into_frames(filename, fps)

Done


## Cartoonize Images

In [5]:
import subprocess
import sys

In [6]:
subprocess.run(["python", "cartoonize.py"])

CompletedProcess(args=['python', 'cartoonize.py'], returncode=0)

## Stitch the frames into a video

In [7]:
from os.path import isfile, join

In [8]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [atoi(c) for c in re.split(r'(\d+)', text)]

In [9]:
def frames_to_video(filename_output, fps, pathIn=join("videos","cartoonized_frames")):
    
    pathOut = os.path.join("videos", "video_output", filename_output+"_"+str(fps)+".avi")
    
    frame_array = []
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
    #for sorting the file names properly
    files.sort(key=natural_keys)
    
    for i in tqdm(range(len(files))):
        filename = os.path.join(pathIn, files[i])
        #reading each files
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)

        #inserting the frames into an image array
        frame_array.append(img)
        
    out = cv2.VideoWriter(pathOut, cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
    
    for i in tqdm(range(len(frame_array))):
        # writing to a image array
        out.write(frame_array[i])
    out.release()

In [10]:
frames_to_video(filename+"_orig_output", fps, join("videos", "frame_split_output"))

100%|█████████████████████████████████████████████████████████████████████████████████| 79/79 [00:00<00:00, 166.65it/s]


In [11]:
frames_to_video(filename+"_cartoonized_output", fps)

100%|█████████████████████████████████████████████████████████████████████████████████| 79/79 [00:00<00:00, 139.06it/s]


## Extract and add Audio

In [ ]:
# extract audio
!ffmpeg -i yourvideo.avi -f mp3 -ab 192000 -vn sound.mp3

In [55]:
# add audio
!ffmpeg -i ocean_cartoonized_output_25.avi -i ocean_sound.mp3 -c copy -map 0:v:0 -map 1:a:0 ocean_cartoonized_output_25_sound.avi

'ffmpeg' is not recognized as an internal or external command,
operable program or batch file.
